In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [2]:
df = pd.read_csv('../data/gutenberg_paragraphs.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

#drop all but the first 1000 paragraphs
#df = df[:100]

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

authors = df['Author'].unique()
author2idx = {author: idx for idx, author in enumerate(authors)}
idx2author = {idx: author for idx, author in enumerate(authors)}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        
        self.labels = [author2idx[author] for author in df['Author']]
        print(self.labels)
        self.texts = [ tokenizer(paragraph, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for paragraph in df['Paragraph']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [4]:
#https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, len(authors))
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [5]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    #use_cuda = False
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [11]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    #use_cuda = False
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [7]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

2687 336 336


In [8]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[4, 0, 1, 2, 0, 1, 0, 0, 1, 4, 0, 2, 1, 3, 0, 1, 2, 3, 1, 4, 0, 1, 1, 4, 4, 2, 0, 2, 0, 0, 0, 1, 2, 0, 3, 1, 0, 2, 4, 1, 3, 1, 1, 1, 0, 0, 1, 2, 0, 3, 2, 3, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 3, 1, 0, 2, 1, 1, 1, 0, 1, 3, 4, 3, 0, 2, 1, 2, 1, 1, 3, 2, 1, 1, 0, 4, 2, 0, 0, 1, 0, 1, 4, 2, 3, 1, 1, 4, 0, 0, 2, 1, 0, 2, 1, 1, 3, 2, 0, 3, 0, 1, 3, 2, 1, 3, 1, 2, 1, 2, 0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 1, 3, 0, 1, 1, 1, 1, 1, 3, 0, 0, 4, 4, 1, 1, 3, 3, 1, 0, 0, 0, 2, 1, 4, 1, 1, 0, 1, 1, 0, 1, 0, 0, 4, 1, 1, 1, 0, 1, 0, 4, 1, 0, 1, 0, 3, 0, 2, 1, 2, 1, 1, 3, 3, 2, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 0, 0, 0, 0, 2, 1, 2, 3, 1, 0, 4, 1, 4, 4, 2, 3, 1, 0, 2, 0, 0, 2, 1, 2, 4, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 3, 1, 3, 1, 0, 0, 0, 1, 1, 3, 1, 1, 0, 0, 3, 1, 1, 1, 2, 0, 0, 1, 2, 1, 1, 1, 0, 0, 2, 1, 3, 1, 3, 1, 3, 3, 1, 0, 0, 1, 3, 0, 0, 2, 0, 0, 1, 0, 1, 1, 1, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 0, 1, 0, 0, 2, 2, 2, 0, 1, 2, 3, 0, 0, 2, 0, 1, 0, 1, 3, 1, 1, 0, 

100%|██████████| 1344/1344 [03:18<00:00,  6.78it/s]


Epochs: 1 | Train Loss:  0.685 | Train Accuracy:  0.465 | Val Loss:  0.439 | Val Accuracy:  0.723


100%|██████████| 1344/1344 [03:22<00:00,  6.64it/s]


...

Epochs: 3 | Train Loss:  0.214 | Train Accuracy:  0.866 | Val Loss:  0.183 | Val Accuracy:  0.878


100%|██████████| 1344/1344 [03:32<00:00,  6.33it/s]


Epochs: 4 | Train Loss:  0.136 | Train Accuracy:  0.930 | Val Loss:  0.121 | Val Accuracy:  0.940


100%|██████████| 1344/1344 [03:35<00:00,  6.25it/s]


Epochs: 5 | Train Loss:  0.079 | Train Accuracy:  0.972 | Val Loss:  0.079 | Val Accuracy:  0.964


In [9]:
evaluate(model, df_test)

[1, 0, 0, 1, 3, 0, 1, 1, 0, 1, 3, 0, 4, 3, 4, 1, 0, 2, 1, 1, 2, 0, 2, 3, 3, 0, 0, 1, 0, 0, 1, 3, 3, 2, 0, 1, 3, 0, 1, 2, 3, 1, 4, 3, 1, 0, 4, 1, 2, 2, 1, 0, 0, 2, 0, 2, 0, 0, 4, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 3, 1, 0, 1, 1, 2, 0, 1, 0, 3, 0, 1, 1, 2, 2, 1, 1, 1, 4, 0, 2, 0, 4, 4, 0, 0, 1, 1, 1, 0, 1, 3, 2, 1, 1, 0, 0, 3, 0, 3, 0, 0, 2, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 3, 1, 0, 0, 3, 1, 0, 4, 0, 3, 2, 1, 0, 0, 0, 2, 2, 1, 1, 2, 1, 0, 4, 3, 0, 1, 2, 1, 0, 1, 1, 3, 1, 1, 0, 1, 3, 1, 1, 0, 0, 0, 0, 1, 3, 0, 2, 1, 3, 4, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 1, 3, 1, 1, 1, 0, 1, 1, 1, 1, 1, 4, 1, 1, 0, 1, 1, 0, 1, 4, 2, 3, 1, 4, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 1, 2, 2, 3, 0, 1, 0, 1, 2, 0, 2, 0, 1, 0, 0, 1, 1, 3, 4, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 4, 2, 3, 3, 0, 0, 0, 3, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 3, 0, 1, 1, 3, 4, 3, 3, 1, 0, 0, 2, 0, 3, 0, 0, 3, 0, 0, 1, 1, 4, 0, 0, 0, 0, 2, 0, 1, 1, 0, 3, 1, 0, 1, 1, 3, 1, 0, 3, 3, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 

In [21]:
#save model
torch.save(model.state_dict(), 'model.pt')